In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

campaign_data = pd.read_csv("C:/Users/sudhe/Desktop/Internship-FUTURENSE/US_Pathway/cl_campaign.csv")

# Preprocessing: Handle missing values
campaign_data.fillna(0, inplace=True)
campaign_data['platform'] = campaign_data['platform'].astype(str)
campaign_data['adset_name'] = campaign_data['adset_name'].astype(str)

# Features and target variable
X = campaign_data[['impressions', 'clicks', 'click_through_rate', 'leads', 'platform', 'adset_name']]
y = campaign_data['total_spent']

# Preprocessing and modeling pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), ['impressions', 'clicks', 'click_through_rate', 'leads']),
        ('cat', OneHotEncoder(), ['platform', 'adset_name'])
    ])),
    ('regressor', RandomForestRegressor())
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

def get_user_input():
    platforms = input('Enter target platforms (comma-separated, e.g., Google,Facebook,LinkedIn): ').split(',')
    
    input_data = {}
    for platform in platforms:
        platform = platform.strip()
        impressions = int(input(f'Enter target impressions for {platform}: '))
        clicks = int(input(f'Enter target clicks for {platform}: '))
        ctr = float(input(f'Enter target click-through rate (CTR) for {platform}: '))
        leads = int(input(f'Enter target leads for {platform}: '))
        adset_name = input(f'Enter new adset name for {platform}: ')
        
        input_data[platform] = {
            'impressions': impressions,
            'clicks': clicks,
            'click_through_rate': ctr,
            'leads': leads,
            'adset_name': adset_name
        }
    
    return input_data

def predict_spending(pipeline, input_data):
    results = {}
    for platform, data in input_data.items():
        input_df = pd.DataFrame({
            'impressions': [data['impressions']],
            'clicks': [data['clicks']],
            'click_through_rate': [data['click_through_rate']],
            'leads': [data['leads']],
            'platform': [platform],
            'adset_name': [data['adset_name']]
        })
        predicted_spending = pipeline.predict(input_df)[0]
        results[platform] = predicted_spending
    return results

# Get user input and predict spending
input_data = get_user_input()
predicted_spending = predict_spending(pipeline, input_data)

# Output the predicted spending
for platform, spending in predicted_spending.items():
    print(f'Predicted Spending for {platform}: {spending}')


Predicted Spending for Google: 14891.170599999994
Predicted Spending for LinkedIn: 4750.8167


In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Calculate MAE, MSE, and R²
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R² Score: {r2}')


Mean Absolute Error (MAE): 181.76577373927054
Mean Squared Error (MSE): 303107.04540574405
R² Score: 0.873800755125757


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import yfinance as yf

# Load the campaign dataset
campaign_data = pd.read_csv("cl_campaign.csv")

# Preprocessing: Handle missing values
campaign_data.fillna(0, inplace=True)
campaign_data['platform'] = campaign_data['platform'].astype(str)
campaign_data['adset_name'] = campaign_data['adset_name'].astype(str)

# Features and target variable
X = campaign_data[['impressions', 'clicks', 'click_through_rate', 'leads', 'platform', 'adset_name']]
y = campaign_data['total_spent']

# Preprocessing and modeling pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), ['impressions', 'clicks', 'click_through_rate', 'leads']),
        ('cat', OneHotEncoder(), ['platform', 'adset_name'])
    ])),
    ('regressor', RandomForestRegressor())
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

def get_user_input():
    platforms = input('Enter target platforms (comma-separated, e.g., Google,Facebook,LinkedIn): ').split(',')
    
    input_data = {}
    for platform in platforms:
        platform = platform.strip()
        impressions = int(input(f'Enter target impressions for {platform}: '))
        clicks = int(input(f'Enter target clicks for {platform}: '))
        ctr = float(input(f'Enter target click-through rate (CTR) for {platform}: '))
        leads = int(input(f'Enter target leads for {platform}: '))
        adset_name = input(f'Enter new adset name for {platform}: ')
        
        input_data[platform] = {
            'impressions': impressions,
            'clicks': clicks,
            'click_through_rate': ctr,
            'leads': leads,
            'adset_name': adset_name
        }
    
    return input_data

def predict_spending(pipeline, input_data):
    results = {}
    for platform, data in input_data.items():
        input_df = pd.DataFrame({
            'impressions': [data['impressions']],
            'clicks': [data['clicks']],
            'click_through_rate': [data['click_through_rate']],
            'leads': [data['leads']],
            'platform': [platform],
            'adset_name': [data['adset_name']]
        })
        predicted_spending = pipeline.predict(input_df)[0]
        results[platform] = predicted_spending
    return results

def fetch_last_market_price(stock_name):
    # Fetching historical market data
    data = yf.Ticker(stock_name).history(period="1mo")
    last_market_price = data['Close'].iloc[-1]
    print("Last market price for", stock_name, ":", last_market_price)
    return last_market_price

def is_investment_feasible(investment_amount, last_market_price):
    # Determine feasibility based on investment amount
    if investment_amount <= last_market_price:
        print("Investment is feasible.")
    else:
        print("Investment is not feasible.")

# Get user input for campaign spending predictions
input_data = get_user_input()
predicted_spending = predict_spending(pipeline, input_data)

# Output the predicted spending
for platform, spending in predicted_spending.items():
    print(f'Predicted Spending for {platform}: {spending}')

# Get user input for stock investment
stock_name = input("Enter share name (ticker symbol, e.g., AAPL for Apple): ")
last_market_price = fetch_last_market_price(stock_name)

# Get investment amount from the user
investment_amount = float(input("Enter investment amount: "))
is_investment_feasible(investment_amount, last_market_price)
